# Maestría en Ciencia de Datos e Inteligencia Artificial
## Módulo: 09: Minería de Datos
### 2025

### *Msc Renzo Claure*
---

Inicializar una sesión de Spark

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("PySpark en Jupyter") \
    .getOrCreate()

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
spark

Importar datos

In [ ]:
df = spark.read.csv("PRACTICAS PYTHON/autos_2.csv", header=True, inferSchema=True, sep=",")

In [ ]:
df.show(1)

In [ ]:
df.select(['normalized-losses', 'compression-ratio']).show(3)

In [ ]:
print(f'El número de filas es: {df.count()} y el número de columnas es: {len(df.columns)}' )

### Tratamiento de datos nulos

In [ ]:
from pyspark.sql.functions import col, sum

In [ ]:
nulls  = df.select(*[sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])
nulls.show()

In [ ]:
import pandas as pd

In [ ]:
nulls.toPandas().transpose()

In [ ]:
#Declarar nulos
df = df.replace('?', None)

In [ ]:
df.printSchema()

In [ ]:
df.select('normalized-losses').show(5)

In [ ]:
df.count()

In [ ]:
#Borrado de registros
df.na.drop().count()

#### Rellenar valores faltantes con un valor específico (por ejemplo, 0)

In [ ]:
df_rellenado = df.na.fill(0)

# Rellenar valores faltantes con valores específicos para cada columna
df_rellenado_especifico = df.na.fill({
    "city-mpg": 0,
    "num-of-cylinders": 4,
    "horsepower": 100,
    "normalized-losses": 100
})

# Mostrar las primeras filas del DataFrame rellenado
df_rellenado_especifico.select("normalized-losses").show(5)

#### Rellenar datos perdidos con la media, mediana

In [ ]:
from pyspark.sql.functions import col, mean, median

In [ ]:
cols = ['price', 'normalized-losses']

In [ ]:
#Revisar este codigo... POr que no funciona???
df_medias = df.select(mean('price').alias('mean_price'), mean('normalized-losses').alias('mean_nl'))
df_medias.show

El cambio aún no se ha realizado se puede comprobar ejecutando:  
> df_medias[0][0]  
> Column<'mean_price[0]'>
> 
Esto es por que aún no se ha realizado la acción que va a concretar la transformación, es necesario aplicar un collect

In [ ]:
#Es necesario hacer un collect
mean_price = df_medias.collect()[0][0]
mean_nl = df_medias.collect()[0][1]

In [ ]:
#Mejor opcion para grandes datasets
mean_nl = df.select(mean(col("normalized-losses"))).first()[0]
mean_nl

In [ ]:
#Cambiar por la mediana
median_ml = df.select(median(col('normalized-losses'))).first()[0]
median_ml

In [ ]:
#reemplazamos los valores perdidos
df_rellenado_especifico = df.select('normalized-losses').na.fill({'normalized-losses': median_ml})
df_rellenado_especifico.select('normalized-losses').show(5)

#### Rellenar valores copn la moda

In [ ]:
df.select('num-of-doors').filter(col('num-of-doors').isNull()).show()

In [ ]:
from pyspark.sql.functions import desc
moda_doors = (
    df.groupBy('num-of-doors') #agrupar por la columna
    .count() #frecuencias
    .orderBy(desc('count')) #ordenar de mayor a menor frecuencia
    .select('num-of-doors')
    .first()[0]
)
moda_doors

In [ ]:
#rellenar valores nulos en la columna "num-of-doors" con la moda
df.na.fill({"num-of-doors": moda_doors}).select('num-of-doors').filter(col('num-of-doors').isNull()).show()

In [ ]:
# Calcular la media, mediana y moda para varias columnas
mean_cmpg = df.select(mean(col("city-mpg"))).collect()[0][0]
median_weight = df.approxQuantile("curb-weight", [0.5], 0.01)[0]
moda_ncyls = (
    df.groupBy("num-of-cylinders")
    .count()
    .orderBy(desc("count"))
    .select("num-of-cylinders")
    .first()[0]
)

# Rellenar valores nulos en múltiples columnas
df_rellenado = df.na.fill({
    "city-mpg": mean_cmpg,  # Rellenar con la media
    "curb-weight": median_weight,  # Rellenar con la mediana
    "num-of-cylinders": moda_ncyls  # Rellenar con la moda
})

# Mostrar el DataFrame con valores nulos rellenados
df_rellenado.select(['city-mpg', 'curb-weight', 'num-of-cylinders' ]).show(5)

**Ejercicio**  
Realice la correccion de todos los datos perdidos del set de datos.
Recomendación, si requiere cambiar de tipo de dato use:  
> from pyspark.sql.types import IntegerType, DoubleType  
> df.withColumn("city-mpg", col("city-mpg").cast(DoubleType())) #si requiere numeros reales  
> df.withColumn("num-of-cylinders", col("num-of-cylinders").cast(IntegerType())) #si requiere enteros

### Transformación de variables

#### Comvertir categorías a números

In [ ]:
from pyspark.ml.feature import StringIndexer

In [ ]:
indexer = StringIndexer(inputCol="fuel-type", outputCol="fuel-index")
indexed_df = indexer.fit(df).transform(df)
indexed_df.show()

In [ ]:
indexer = StringIndexer(inputCol='fuel-type', outputCol='fuel-index')
indexed_df = indexer.fit(df).transform(df)
indexed_df.select(['fuel-type', 'fuel-index']).show(5)

In [ ]:
indexed_df.groupby(['fuel-type', 'fuel-index']).count().show()

#### Aplicar One Hot Encoding

In [ ]:
from pyspark.ml.feature import OneHotEncoder

encoder = OneHotEncoder(
    inputCols=["fuel-index"],
    outputCols=["fuel-encoded"],
    dropLast=False  #incluye todas las categorías
)

encoded_df = encoder.fit(indexed_df).transform(indexed_df)

In [ ]:
encoded_df.select(['fuel-type', 'fuel-encoded']).filter(col('fuel-type')=='diesel').show(3)

In [ ]:
encoded_df.select(['fuel-type', 'fuel-encoded']).show(3)

#### Cerrar la sesión de Spark

In [ ]:
spark.stop()